In [ ]:
# Clone and install axonet
import os
import sys
from pathlib import Path
from IPython import get_ipython

repo_url = 'https://github.com/gileshall/axonet.git'
repo_dir = Path('/content/axonet')
if not repo_dir.exists():
    get_ipython().system(f'git clone -q {repo_url} {repo_dir}')

get_ipython().run_line_magic('cd', str(repo_dir))
get_ipython().system('pip -q install -e .')

get_ipython().run_line_magic('cd', '/content')
sys.path.append(str(repo_dir))



In [ ]:
# Imports and helper functions
import re
import json
import requests
import numpy as np
import trimesh
from ipywidgets import VBox, HBox, Dropdown, Button, HTML, Output, IntSlider, Checkbox, Text
from IPython.display import display

from axonet.axonet.io import load_swc
from axonet.axonet.visualization.mesh import MeshRenderer

BASE_URL = 'https://download.brainimagelibrary.org/biccn/zeng/pseq/morph/200526/'

def list_remote_swcs() -> list:
    try:
        r = requests.get(BASE_URL, timeout=30)
        text = r.text
        ids = re.findall(r'href="(\d+)_transformed\.swc"', text)
        unique_ids = sorted({int(x) for x in ids})
        options = [(str(i), f"{BASE_URL}{i}_transformed.swc") for i in unique_ids]
        return options if options else [("No samples found", "")]
    except Exception as e:
        print(f"Error fetching sample list: {e}")
        return [("Error loading samples", "")]

remote_dropdown = Dropdown(
    options=list_remote_swcs(),
    description='Sample',
    layout={"width": "350px"}
)

segments_slider = IntSlider(value=24, min=6, max=64, step=2, description='Segments', continuous_update=False)
cap_checkbox = Checkbox(value=False, description='Cap ends')
translate_checkbox = Checkbox(value=True, description='Origin at soma')
colorize_checkbox = Checkbox(value=True, description='Colorize by class')

load_button = Button(description='Load & Generate', button_style='primary')
status = HTML(value='<i>Select a sample and click "Load & Generate"</i>')
controls_out = Output()

with controls_out:
    display(HBox([remote_dropdown, load_button]))
    display(HBox([segments_slider, cap_checkbox, translate_checkbox, colorize_checkbox]))
    display(status)

display(controls_out)

scene_container = Output()
display(scene_container)



In [ ]:
# Render handler using trimesh notebook viewer
from trimesh.viewer import notebook as tnb

_download_cache = {}

def _download_swc(url: str, dest: Path) -> Path:
    if url in _download_cache:
        return _download_cache[url]
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()
    dest.parent.mkdir(parents=True, exist_ok=True)
    with open(dest, 'wb') as f:
        f.write(resp.content)
    _download_cache[url] = dest
    return dest


def on_click_load(_):
    status.value = '<i>Downloading SWC...</i>'
    url = remote_dropdown.value
    if not url:
        status.value = '<b>No SWC selected.</b>'
        return
    local = Path('/content/data') / Path(url).name
    try:
        swc_path = _download_swc(url, local)
        status.value = '<i>Parsing SWC...</i>'
        neuron = load_swc(str(swc_path), validate=True)
        renderer = MeshRenderer(neuron)
        status.value = '<i>Building scene...</i>'
        scene = renderer.build_scene(
            segments=int(segments_slider.value),
            cap=bool(cap_checkbox.value),
            translate_to_origin=bool(translate_checkbox.value),
            glue_union=False,
            colorize=bool(colorize_checkbox.value)
        )
        status.value = '<i>Rendering...</i>'
        html = tnb.scene_to_html(scene, escape_quotes=False)
        with scene_container:
            scene_container.clear_output()
            display(HTML(html))
        status.value = '<b>Done.</b>'
    except Exception as e:
        status.value = f'<b>Error:</b> {e}'

load_button.on_click(on_click_load)

